# 云分类器

In [1]:
import pickle
import os

In [2]:
import torch
from torchvision import transforms, datasets, models

## 载入数据

In [3]:
train_data_path = r"./data"
test_data_path = r"./test"

In [4]:
batch_size = 16

In [5]:
data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(),
    transforms.ToTensor()
])

In [6]:
cloud_datasets = datasets.ImageFolder(train_data_path,transform=data_transform)

In [7]:
image_loader = torch.utils.data.DataLoader(cloud_datasets,batch_size=batch_size,shuffle=True,num_workers=4)

## 构造模型

In [8]:
classifier = models.resnet50(num_classes=10)

In [9]:
model_path = r"./model"

In [10]:
model_name = r"model.pickle"

In [22]:
try:
    with open(os.path.join(model_path,model_name),"rb") as f:
        state_dict = pickle.load(f)
        classifier.load_state_dict(state_dict)
except:
    print("can't load model")

##  训练模型

### 选择loss和优化器

In [12]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(classifier.parameters())

### 定义训练流程

In [13]:
def train(model, loss, optimizer, data_loader, epochs = 1, cuda = True):
    if cuda:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0.0
        data_sizes = 0.0
        
        for i,data in enumerate(data_loader):
            inputs, labels = data
            if cuda:
                inputs, labels = inputs.to(device), labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            #print(outputs)
            preds = torch.argmax(outputs, dim=1)
            
            loss_result = loss(outputs, labels)
            loss_result.backward()
            optimizer.step()
            
            running_loss += loss_result.item()
            running_corrects += torch.sum(preds == labels.data)
            data_sizes += len(labels)
            
        print('[%d, %5d] loss: %.3f, acc: %.3f' %
          (epoch + 1, data_sizes, running_loss / data_sizes, running_corrects.item() / data_sizes))

### 训练模型

In [20]:
train(classifier,loss, optimizer, image_loader, epochs=10)

[1,   324] loss: 0.039, acc: 0.772
[2,   324] loss: 0.039, acc: 0.793
[3,   324] loss: 0.037, acc: 0.840
[4,   324] loss: 0.036, acc: 0.793
[5,   324] loss: 0.039, acc: 0.796
[6,   324] loss: 0.036, acc: 0.836
[7,   324] loss: 0.036, acc: 0.855
[8,   324] loss: 0.039, acc: 0.806
[9,   324] loss: 0.031, acc: 0.840
[10,   324] loss: 0.032, acc: 0.849


### 保存模型

In [21]:
with open(os.path.join(model_path,model_name),"wb") as f:
    device = torch.device("cpu")
    classifier.to(device)
    state_dict = classifier.state_dict()
    pickle.dump(state_dict,f)